In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Recarregar base
df = pd.read_csv("fermentacao_etanol.csv")

In [ ]:
# Limpar base como feito anteriormente
df = df.dropna(subset=["Tempo (h)", "Rendimento Etanol (%)", "pH", "Temperatura (°C)", "Glicose (g/L)", "Agitação (rpm)"])
df = df[
    (df["pH"].between(3.5, 6.0)) &
    (df["Glicose (g/L)"] <= 300) &
    (df["Tempo (h)"].between(12, 48))
]

In [ ]:
# Criar variável alvo binária (eficiência >= 50%)
df["Eficiente"] = (df["Rendimento Etanol (%)"] >= 50).astype(int)

In [ ]:
# Definir X e y
X = df[["Tempo (h)", "Temperatura (°C)", "pH", "Glicose (g/L)", "Agitação (rpm)"]]
y = df["Eficiente"]

In [ ]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 1. Árvore de Decisão
dtree = DecisionTreeClassifier(max_depth=4, random_state=42)
dtree.fit(X_train, y_train)
y_pred_tree = dtree.predict(X_test)
acc_tree = accuracy_score(y_test, y_pred_tree)
cm_tree = confusion_matrix(y_test, y_pred_tree)


In [ ]:
# Plotar a árvore
plt.figure(figsize=(14, 8))
plot_tree(dtree, feature_names=X.columns, class_names=["Ineficiente", "Eficiente"], filled=True, rounded=True)
plt.title("Árvore de Decisão - BioFermenta")
plt.show()

In [ ]:
# 2. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
importancias_rf = rf.feature_importances_

In [ ]:
# 3. Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
acc_gb = accuracy_score(y_test, y_pred_gb)
importancias_gb = gb.feature_importances_

In [ ]:
# Comparar acurácias
comparativo = pd.DataFrame({
    "Modelo": ["Árvore de Decisão", "Random Forest", "Gradient Boosting"],
    "Acurácia": [acc_tree, acc_rf, acc_gb]
})

In [ ]:
# Importância das variáveis
importancias_df = pd.DataFrame({
    "Variável": X.columns,
    "Importância - RF": importancias_rf,
    "Importância - GB": importancias_gb
}).sort_values(by="Importância - RF", ascending=False)

# Exibir os resultados no Colab
print("Comparativo de Modelos Baseados em Árvores:")
display(comparativo)

print("Importância das Variáveis (Random Forest vs Gradient Boosting):")
display(importancias_df)